1. Menentukan Library yang digunakan

In [4]:
import pandas as pd 
import matplotlib.pyplot as  plt 
import seaborn as sns 
import missingno as msno 
import sklearn
print(sklearn.__version__)

ModuleNotFoundError: No module named 'matplotlib'

2. Load dataset


In [20]:
data = pd.read_csv("musik.csv")

3. menampilkan info dari dataset

In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 736 entries, 0 to 735
Data columns (total 33 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Timestamp                     736 non-null    object 
 1   Age                           735 non-null    float64
 2   Primary streaming service     735 non-null    object 
 3   Hours per day                 736 non-null    float64
 4   While working                 733 non-null    object 
 5   Instrumentalist               732 non-null    object 
 6   Composer                      735 non-null    object 
 7   Fav genre                     736 non-null    object 
 8   Exploratory                   736 non-null    object 
 9   Foreign languages             732 non-null    object 
 10  BPM                           629 non-null    float64
 11  Frequency [Classical]         736 non-null    object 
 12  Frequency [Country]           736 non-null    object 
 13  Frequ

4. Mengecek dan menangani missing values


In [22]:
# Mengecek missing values
print(data.isnull().sum())

#Mengisi missing values untuk kolom numerik dengan rata-rata
data['Hours per day'] = data['Hours per day'].fillna(data['Hours per day'].mean())

# Drop baris dengan missing values di kolom penting
data = data.dropna(subset=['Fav genre', 'Age', 'Anxiety', 'Depression', 'Insomnia', 'OCD'])

Timestamp                         0
Age                               1
Primary streaming service         1
Hours per day                     0
While working                     3
Instrumentalist                   4
Composer                          1
Fav genre                         0
Exploratory                       0
Foreign languages                 4
BPM                             107
Frequency [Classical]             0
Frequency [Country]               0
Frequency [EDM]                   0
Frequency [Folk]                  0
Frequency [Gospel]                0
Frequency [Hip hop]               0
Frequency [Jazz]                  0
Frequency [K pop]                 0
Frequency [Latin]                 0
Frequency [Lofi]                  0
Frequency [Metal]                 0
Frequency [Pop]                   0
Frequency [R&B]                   0
Frequency [Rap]                   0
Frequency [Rock]                  0
Frequency [Video game music]      0
Anxiety                     

5. Memastikan format data 
    pastikan kolom memiliki tipe data yang benar terutama untuk 
    variabel yang digunakan dalam model

In [23]:
# Pastikan kolom usia dan jam mendengarkan musik numerik
data['Age'] = pd.to_numeric(data['Age'], errors='coerce')
data['Hours per day'] = pd.to_numeric(data['Hours per day'], errors='coerce')

# Pastikan kolom genre adalah kategorikal
data['Fav genre'] = data['Fav genre'].astype('category')

6. Menangani Outlier
    Cek dan tangani data yang tidak masuk akal (terlalu kecil/
    terlalu besar) atau jam mendengarkan musik perhari yang tidak 
    masuk akal

In [24]:
#Hapus outlier pada kolom Age dan Hours per day
data = data[(data['Age'] > 10) & (data['Age'] < 100)]
data = data[(data['Hours per day'] >= 0) & (data['Hours per day'] <= 24)]

6. Encoding untuk Variabel Kategorikal
   Variabel kategorikal seperti Fav genre perlu dikonversi menjadi 
   format numerik atau encoding agar dapat digunakan oleh model 
   machine learning.

In [25]:
from sklearn.preprocessing import OneHotEncoder

# One-hot encoding untuk genre
encoder = OneHotEncoder(handle_unknown='ignore')
encoded_genre = pd.DataFrame(encoder.fit_transform(data[['Fav genre']]).toarray(), columns=encoder.get_feature_names_out(['Fav genre']))

# Gabungkan kembali ke data utama
data = pd.concat([data.reset_index(drop=True), encoded_genre.reset_index(drop=True)], axis=1)

# Drop kolom asli 'Fav genre'
data = data.drop(columns=['Fav genre'])

7. Menyiapkan dataset
    Pisahkan data menjadi fitur (X) dan target (y) untuk model 
    machine learning.

In [26]:
# Fitur
X = data[['Age', 'Hours per day'] + list(encoded_genre.columns)]  # Tambahkan kolom hasil encoding

# Target
y = data[['Anxiety', 'Depression', 'Insomnia', 'OCD']]

7. Menyimpan data yang telah diproses

In [27]:
# Simpan dataset bersih
data.to_csv("cleaned_musik.csv", index=False)